In [2]:
pip install diagrams

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.7/824.7 kB 36.9 MB/s eta 0:00:00


In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
# Import required modules from diagrams
from diagrams import Diagram, Cluster, Edge
from diagrams.gcp.database import SQL
from diagrams.gcp.storage import GCS
from diagrams.onprem.workflow import Airflow
from diagrams.onprem.container import Docker
from diagrams.custom import Custom
from diagrams.programming.language import Python

In [38]:
# Create the diagram with smaller clusters
with Diagram("Architecture Diagram", show=True):
    # Data Ingestion with Docker icon only for GAIA Dataset, Airflow, and GCP Bucket
    with Cluster("Data Ingestion"):
        with Cluster("Docker"):
            docker_icon_ingestion = Docker("Docker")
            gaia_dataset = Custom("GAIA Dataset", "/content/gaia.png")  # Adjust path as needed
            airflow = Airflow("Airflow")
            gcp_bucket = GCS("GCP Bucket")

            # Define relationships within the Docker container
            gaia_dataset >> airflow >> gcp_bucket
            docker_icon_ingestion - Edge(label="runs on") - gaia_dataset

        # Metadata and GCP SQL outside the Docker container
        metadata = Custom("Metadata", "/content/json.png")  # Adjust path as needed
        gcp_sql = SQL("GCP SQL")
        metadata >> gcp_sql

    # API & Client section with smaller clusters
    with Cluster("API & Client"):
        # FastAPI and OpenAI in a Docker environment but in separate clusters
        with Cluster("Docker"):
            docker_icon_api = Docker("Docker")

            # Smaller clusters for FastAPI and OpenAI
            fastapi = Custom("FastAPI", "/content/fastapi.png")  # Adjust path for a different FastAPI image
            openai = Custom("OpenAI", "/content/openai.png")  # Adjust path as needed

            # Connect FastAPI to OpenAI within the Docker environment
            fastapi >> openai
            openai >> fastapi

            # Show that FastAPI and OpenAI run in one Docker container
            docker_icon_api - Edge(label="runs on") - fastapi

        # Streamlit in a separate Docker container with a bi-directional arrow to FastAPI
        with Cluster("Docker"):
            streamlit = Python("Streamlit")
            streamlit << Edge(label="communicates with", forward=False) >> fastapi

        # Docker icon for Streamlit
        docker_icon_streamlit = Docker("Docker")
        docker_icon_streamlit - Edge(label="runs on") - streamlit

    # Connect FastAPI to the Data Ingestion component (using Airflow as a representative)
    fastapi << Edge(label="connects to Data Ingestion") >> airflow